In [ ]:
from IPython.display import HTML
from matplotlib_inline.backend_inline import set_matplotlib_formats

# Use inline backend so CSS scaling works on PNG/SVG outputs
set_matplotlib_formats('png')

HTML("""
<style>
/* =======================
   Matplotlib figure sizing
   ======================= */

/* Images (PNG/retina) */
.jp-RenderedImage img,
.jp-OutputArea-output img {
  width: 100% !important;
  height: auto !important;
  max-width: 100% !important;
  display: block;
  box-sizing: border-box;
}

/* SVG outputs */
.jp-RenderedSVG svg,
.jp-OutputArea-output svg {
  width: 100% !important;
  height: auto !important;
}

/* ipympl / canvas-backed figures (in case %matplotlib widget is used) */
.jupyter-matplotlib canvas.mpl-canvas,
.jp-OutputArea-output canvas.mpl-canvas,
.jp-OutputArea-output canvas {
  width: 100% !important;
  height: auto !important;
  max-width: 100% !important;
}

/* Reset container widths */
.jp-OutputArea-output .matplotlib-figure,
.jp-RenderedHTMLCommon .matplotlib-figure,
.jp-OutputArea-output .output_png {
  width: 100% !important;
}

/* =======================
   ipywidgets styling
   ======================= */

/* Make slider labels bold */
.widget-label {
    font-weight: bold;
}

/* Style ipywidgets buttons */
.jp-OutputArea .widget-button,
button.widget-button {
    font-weight: bold;
    background-color: #1976d2;  /* blue background */
    color: white;               /* white text */
    border: none;
    border-radius: 6px;
    padding: 6px 12px;
    font-size: 14px;
    cursor: pointer;
}

/* Button hover effect */
.jp-OutputArea .widget-button:hover,
button.widget-button:hover {
    background-color: #1259a4;  /* darker blue */
}
</style>
""")

# **The Struggle for Power** 📡⚡  

## **Directing Energy with Precision**  

A radar antenna doesn’t just spray energy into space—it **focuses** it, shaping a beam of microwaves that races outward. As the wavefront expands, it’s like the skin of an inflating balloon: the farther it goes, the thinner the energy becomes. 🎈  This spreading reduces the **power density**—the wave energy available per unit area. 🌍  

## **The Inverse Square Law**  

Energy dilution follows the **inverse square law**: the same power spread over a larger surface means less intensity. 🔢  Think of ripples from a stone dropped in water—large and strong near the splash, but gentler as they travel outward. 🌊  As the wavefront grows in size, its energy is shared across more and more area, so each point receives less: that’s the **inverse-square law** in action.  

The simulation below illustrates this principle:
- **Left:** A circular wavefront expands; its line grows **thinner** and **fainter** with distance (∝ 1/r²). The thickness of the line represents the **power density** of the wave as it expands.  
- **Right:** The theoretical curve shows power density, which is **inversely proportional to the square of the distance**, with a live marker at the current range.  
- **Try it:** Press ▶️ to animate, or move the slider to see how fast the power dilutes with increasing distance.

In [ ]:
def build_inverse_square_voila_app():
    """
    Voilà-friendly inverse-square demo with slider + playback.
    Left: expanding ring with thickness & brightness ∝ 1/r²
    Right: power density ~ 1/(4π r²) with live marker
    """
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import (
        FloatSlider, IntSlider, Play, HBox, VBox, Output, Label, Button, Layout, jslink
    )
    from IPython.display import display

    # --- Parameters ---
    r_min, r_max = 2.0, 40.0
    step = 1.0
    base_thickness = 10.0     # linewidth near r = r_min
    min_thickness = 0.5
    min_alpha = 0.1
    epsilon = 1e-9
    P0 = 1.0

    # Discrete r-values for clean stepping via Play
    r_values = np.arange(r_min, r_max + step, step)
    n_frames = len(r_values)

    # Precompute theory curve for right plot
    r_curve = np.linspace(r_min, r_max, 600)
    power_density = P0 / (4*np.pi*np.maximum(r_curve, epsilon)**2)

    # --- Widgets ---
    r_slider = FloatSlider(
        description="Distance r",
        value=r_min, min=r_min, max=r_max, step=step,
        readout_format=".0f", continuous_update=True, layout=Layout(width="320px")
    )

    # Playback: Play + frame slider (hidden) that maps to r_values
    play = Play(
        value=0, min=0, max=n_frames - 1, step=1, interval=120,  # interval ms/frame
        show_repeat=True
    )
    frame = IntSlider(value=0, min=0, max=n_frames - 1, step=1, layout=Layout(width="320px"))
    # Link Play <-> frame
    jslink((play, "value"), (frame, "value"))

    # Map frame -> r, and keep r_slider in sync
    def on_frame(change):
        idx = change["new"]
        # guard
        if 0 <= idx < n_frames:
            r_slider.value = float(r_values[idx])  # triggers draw via r_slider observer

    frame.observe(on_frame, names="value")

    # Reset button
    reset_btn = Button(description="Reset", layout=Layout(width="90px"))
    def on_reset(_):
        play.value = 0     # jump playhead to start
        play._repeat = False  # ensure single shot if desired; repeat toggle is visible on the Play
        r_slider.value = float(r_values[0])  # triggers draw

    reset_btn.on_click(on_reset)

    out = Output()

    # --- Drawing ---
    def draw_frame(r: float):
        fig = plt.figure(figsize=(9.5, 5))

        # Left: expanding ring (line circle) with thickness & alpha ~ 1/r²
        ax1 = fig.add_subplot(1, 2, 1)
        ax1.set_aspect("equal", adjustable="box")
        lim = r_max * 1.05
        ax1.set_xlim(-lim, lim); ax1.set_ylim(-lim, lim)
        ax1.axis("off")
        ax1.set_title("Expanding Wavefront (thickness & brightness ↓ ~ 1/r²)")
        ax1.plot(0, 0, 'ko', markersize=6)

        theta = np.linspace(0, 2*np.pi, 600)
        x = r*np.cos(theta); y = r*np.sin(theta)

        scale = (r_min / max(r, r_min))**2          # 1/r² scaling
        lw = max(base_thickness * scale, min_thickness)
        alpha = max(scale, min_alpha)
        ax1.plot(x, y, linewidth=lw, color=(0.2, 0.5, 1.0, alpha))

        # Right: inverse-square curve with live marker
        ax2 = fig.add_subplot(1, 2, 2)
        ax2.plot(r_curve, power_density, linewidth=2)
        ax2.set_xlabel("Distance r")
        ax2.set_ylabel("Power density ~ 1/(4π r²)")
        ax2.set_title("Inverse-Square Law")
        ax2.grid(True, linestyle="--", linewidth=0.5)
        y_now = P0 / (4*np.pi*max(r, epsilon)**2)
        ax2.axvline(r, linestyle=":", linewidth=1.5)
        ax2.plot([r], [y_now], 'o')
        ax2.text(0.02, 0.95,
                 f"r = {r:.0f}\nThickness ∝ 1/r²\nBrightness ∝ 1/r²",
                 transform=ax2.transAxes, ha="left", va="top")

        plt.tight_layout()
        with out:
            out.clear_output(wait=True)
            display(fig)
        plt.close(fig)

    # --- Observers ---
    def on_slider(change):
        # Keep frame slider roughly in sync (nearest index)
        r_new = change["new"]
        idx = int(np.clip(np.round((r_new - r_min) / step), 0, n_frames - 1))
        if frame.value != idx:
            frame.value = idx  # this will also update Play via jslink
        draw_frame(r_new)

    r_slider.observe(on_slider, names="value")

    # Initial render
    draw_frame(r_values[0])

    # Layout
    # Show Play control prominently; keep the numeric frame slider visible for manual scrubbing.
    controls_row = HBox([play, frame, r_slider, reset_btn], layout=Layout(align_items="center"))
    help_label = Label("Use Play ▶ to animate, or scrub with the slider: ring fades & thins with 1/r².")
    return VBox([controls_row, help_label, out])

# Build & display
from IPython.display import display
inverse_square_law_app = build_inverse_square_voila_app()
display(inverse_square_law_app)

## **The Challenge of Weak Echoes** 🌫️🔈  

We've got some bad news: the simulation above only illustrated a **one-way** propagation. In reality, radar signals have to **go out and then come back** after bouncing off the target.  

Now picture that return journey 🛰️↔️🌍: only a **tiny fraction** of the transmitted energy reflects back, and it has to travel the long path through space and atmosphere a second time. By the time it arrives, the echo is **vanishingly small** — like trying to hear a **whisper across a noisy stadium** 🎤👂🏟️.  

And yet… modern radars can still pick out these faint signals 🔎✨ and turn them into precise, detailed measurements. That’s the power of radar engineering! ⚡📡   

## **Radar Cross Section (RCS)**  

Not every target reflects equally. The **Radar Cross Section (RCS)** measures how much of the radar wave a target bounces back:  

- A **metallic structure** pointing the right way? Strong reflector. 🏗️  
- A **patch of calm water at an angle**? Weak reflector. 🌊  

An RCS of one square meter means the target reflects like an **ideal metal sphere of that size**. RCS depends on material, size, shape, wavelength, and viewing angle. 🎯  

## 📡 **The Radar Range Equation**  

The **Radar Range Equation** ties all this together. It explains how transmitted power weakens and changes on its round trip, and what finally reaches the receiver. These are the most crucial factors:  

- **Transmit Power** – How much energy we send out ⚡  
- **Antenna Size** – How tightly the beam is focused 📡  
- **Range (Distance)** – Power weakens with the **square of distance—twice** 📏  
- **Radar Cross Section (RCS)** – How much the target reflects back 🎯  

Because the wave must spread out **on the way there and back**, the received power drops with the **fourth power of range**. In other words, the inverse-square law hits us **twice as hard**. ⚡📉  

## **The Extreme Challenge of Spaceborne SAR**  

For satellites, the struggle is extreme. They can’t carry **huge antennas** or **giant transmitters**, yet must sense echoes from **hundreds of kilometers away**.  

The received power is often **20 orders of magnitude smaller** than what was sent! ❌ And still, clever engineering captures those whispers and turns them into **sharp images of Earth’s surface**. 🌍  

## ⚡ **Signal Power, Amplitude, and Noise**  

The radar “hears” the returning waves as tiny **voltages** induced in its antenna. The **signal power** is proportional to the **square of the amplitude**—a stronger oscillation means more power. 📈  

But these signals don’t arrive alone. They must fight against something called **noise**. By *noise* we mean any electromagnetic radiation that are **not part of the target echo** but still get recorded by the receiver. 🎧❌  

Major noise sources include:  
- 🌍✨ **Thermal radiation** from Earth, atmosphere, and even deep space  
- 🔥 **The radar’s own electronics**, which inevitably generate random fluctuations  

And one key factor that controls how much of this noise we collect is the **receiver bandwidth** ⚠️. The wider the receiver “listens,” the more of the background noise from these sources gets integrated into the measurement. The result is a constant tug-of-war ⚖️ between **signal and noise**—and radar engineers work hard to make sure the signal wins. 🏆  

## ⚖️ **Resolution vs. Noise**  

Sharper radar images demand **a higher bandwidth** for the chirp we transmit. But a wider bandwidth also lets in **more noise**. So improving resolution means fighting a noisier background. In radar design, every choice is a balancing act—strengthening the signal while managing its weaknesses. 🎭  

Now that we’ve seen how radar signals weaken on their long journey out and back, let’s bring it all together with the **Radar Range Equation**. 📡✨  

This equation tells us how the maximum range of a radar depends on key factors like **transmit power, antenna size, collection time, and target reflectivity (RCS)**. By adjusting these parameters below, you can see how each one shapes the radar’s ability to detect faint echoes across vast distances. 🚀  

The **maximum range** in the plot below marks the farthest distance at which a target’s echo still rises just above the noise level—meaning the radar can still detect it. 📡✨ Beyond this limit, the echoes become so faint that they are buried in background noise 🌫️🔈, and the target disappears from view.  

In [ ]:
def build_range_equation_voila_app():
    """
    Voilà-friendly Radar Range Equation explorer.
    Adjustable sliders: antenna width (m), antenna length (m), transmit power (W), RCS (m^2),
    bandwidth (MHz), pulse duration (s).
    Returns a VBox you can display: `display(build_range_equation_voila_app())`
    """
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import FloatSlider, FloatLogSlider, VBox, HBox, Label, Output, Button, Layout

    # ---- Constants (fixed) ----
    k = 1.38e-23      # Boltzmann's constant (J/K)
    c = 3e8           # Speed of light (m/s)
    T = 290           # Receiver noise temperature (K)
    F = 10**(5/10)    # Noise figure (5 dB)
    L = 1.0           # System losses
    SNR_min = 10      # Minimum SNR for detection (linear)
    freq = 9.65e9     # Radar frequency (Hz)
    lamb = c / freq   # Wavelength (m)

    PRF = 5e3                 # Pulse repetition frequency (Hz)
    default_pulse = 2e-5      # Pulse duration (s)
    default_bw_mhz = 1200.0   # Bandwidth (MHz)

    # Time axis for the plot
    t_vals = np.linspace(0.1, 10.0, 200)  # collection duration in seconds
    N_pulses = PRF * t_vals               # total pulses

    # ---- Widgets ----
    w_width = FloatSlider(
        description="Antenna W (m)",
        value=3.2, min=0.2, max=6.0, step=0.05, readout_format=".2f", continuous_update=False
    )
    w_width.style.description_width = 'initial'

    w_length = FloatSlider(
        description="Antenna L (m)",
        value=0.4, min=0.2, max=3.0, step=0.05, readout_format=".2f", continuous_update=False
    )
    w_length.style.description_width = 'initial'

    w_pt = FloatSlider(
        description="Tx Power (W)",
        value=2000.0, min=1.0, max=10000.0, step=1.0, readout_format=".0f", continuous_update=False
    )
    w_pt.style.description_width = 'initial'

    w_sigma = FloatSlider(
        description="RCS (m²)",
        value=1.0, min=0.001, max=100.0, step=0.001, readout_format=".3f", continuous_update=False
    )
    w_sigma.style.description_width = 'initial'

    # Display bandwidth in MHz for a clean readout; convert to Hz in compute
    w_bw_mhz = FloatSlider(
        description="Bandwidth (MHz)",
        value=default_bw_mhz, min=1.0, max=2000.0, step=1.0, readout_format=".0f", continuous_update=False
    )
    w_bw_mhz.style.description_width = 'initial'

    w_pulse = FloatLogSlider(
        description="Pulse (s)", base=10, min=-7, max=-3, step=0.1,
        value=default_pulse, continuous_update=False, readout_format=".2e",
        layout=Layout(width="48%")
    )
    w_pulse.style.description_width = 'initial'

    reset_btn = Button(description="Reset", layout=Layout(width="100px"))
    out = Output()

    def compute_rmax_km(antenna_width, antenna_length, Pt, sigma, pulse_s, bw_hz):
        # Simple rectangular-χ matched filter processing gain approximation
        G_linear = (4 * np.pi * antenna_width * antenna_length) / (lamb ** 2)
        G_chirp  = bw_hz * pulse_s                      # time–bandwidth product
        numerator = Pt * (G_linear ** 2) * (lamb ** 2) * sigma * G_chirp
        denominator = (4 * np.pi) ** 3 * k * T * bw_hz * F * L * SNR_min
        R_max = ((numerator * N_pulses) / denominator) ** 0.25
        return R_max / 1e3  # km

    def update(*_):
        with out:
            out.clear_output(wait=True)

            bw_hz = w_bw_mhz.value * 1e6
            R_km = compute_rmax_km(
                w_width.value, w_length.value, w_pt.value, w_sigma.value,
                w_pulse.value, bw_hz
            )

            plt.figure(figsize=(7, 5))
            plt.plot(t_vals, R_km)
            plt.xlabel("Collection Duration (s)")
            plt.ylabel("Maximum Range (km)")
            plt.title("Maximum SAR Range vs. Collection Duration")
            plt.grid(True, linestyle="--", linewidth=0.5)

            # Small annotation block with current params
            text = (
                f"W={w_width.value:.2f} m, L={w_length.value:.2f} m, "
                f"Pt={w_pt.value:.0f} W, σ={w_sigma.value:.3f} m²\n"
                f"f={freq/1e9:.2f} GHz, B={w_bw_mhz.value:.0f} MHz, "
                f"τ={w_pulse.value*1e6:.1f} µs, PRF={PRF/1e3:.1f} kHz"
            )
            plt.annotate(
                text, xy=(0.02, 0.98), xycoords="axes fraction",
                va="top", ha="left", fontsize=9,
                bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="0.7", alpha=0.9)
            )
            plt.show()

    def do_reset(_):
        w_width.value  = 3.2
        w_length.value = 0.4
        w_pt.value     = 2000.0
        w_sigma.value  = 1.0
        w_bw_mhz.value = default_bw_mhz
        w_pulse.value  = default_pulse

    # Wire up events (include pulse & bandwidth!)
    for w in (w_width, w_length, w_pt, w_sigma, w_bw_mhz, w_pulse):
        w.observe(update, names="value")
    reset_btn.on_click(do_reset)

    # Initial draw
    update()

    controls = VBox([
        HBox([w_width, w_length]),
        HBox([w_pt, w_sigma, reset_btn]),
        HBox([w_bw_mhz, w_pulse]),
    ])
    ui = VBox([controls, out])
    return ui

from IPython.display import display
range_simulation = build_range_equation_voila_app()
display(range_simulation)

### 📊 Why Bandwidth Doesn’t Change Maximum Range  

When you play with the range equation simulation, you’ll notice something odd: changing the **chirp bandwidth** doesn’t change the **maximum range**.  

Why? Because two effects **cancel each other out**:  
- A wider bandwidth means **more noise** sneaks into the receiver 🎶  
- But the same wider bandwidth also gives a **processing gain** when we compress the chirp ⏱️  

Result: the bandwidth **drops out of the equation**. ❌  We'll see later that the **increased noise** does affect how our SAR images look though.

---

### 🌟 Understanding Noise-Equivalent Sigma Zero (NESZ)  

In SAR, we’re not looking for single point targets — we’re mostly imaging the **ground surface**. 🌍  Each pixel in the image acts like a **tiny mirror** 🪞 that reflects some of the incoming radar energy back.  

---

### 🔍 Higher Resolution = Smaller Mirrors  

- Improve the resolution → each pixel represents a **smaller patch of ground**, i.e. a smaller mirror 📏  
- Smaller patch → less reflected energy per pixel 📉  

So, while the **total reflected energy from the surface stays the same**, each pixel now gets a smaller “slice” of it.  

To make comparisons fair, we use **normalized backscatter**, also called **sigma nought** (σ⁰) by radar engineers: we take the measured echo from a pixel and **divide by the pixel’s area**. This way, the values describe a **true physical property of the surface**, independent of pixel size. ✅  

---

### 🔑 Noise-Equivalent Sigma Zero (NESZ)  

NESZ tells us:  
**“How strong would the normalized backscatter have to be for its echo to match the noise level of the radar?”**  

- 📉 **Lower NESZ** → radar is sensitive, can detect weak backscatter  
- 📈 **Higher NESZ** → only strong reflectors stand out  

---

### 🎉 The Party Conversation  

Imagine you’ve just walked into a **crowded party** 🥂. The room is buzzing — music is pumping, people are laughing, and glasses are clinking.  

Across the room, your friend is trying to get your attention:  
- 🗣️ Their **voice** is the **signal** we are interested to capture.  
- 🎶 The music and chatter around you are the **noise**.  

At first, the background hum is manageable, and even across the room you can catch your friend's words. But as the DJ turns the volume up and more people join in, the crowd gets louder. Now your friend has to **raise their voice** just to reach you.  

In this example, the **NESZ** would describe the **minimum “volume” your friend must speak at** in order for their words not to drown in the crowd.  

- If the party is quiet (low NESZ), even a soft voice carries across the room. ✔️  
- If the party is noisy (high NESZ), only shouting works — and even then, it might not be enough. ❌  

👉 In radar terms, NESZ tells us how strong the reflected signal from the ground needs to be before it stands out against the noise floor.  

### ☕ A Quieter Room vs. a Loud Club  

Now imagine you and your friend step out to a **quiet café** ☕ instead. Suddenly, even across the table, their words are perfectly clear. Why? Because the **background noise is lower**.  

That’s what a well-designed radar system does:  
- Better antennas, lower-noise receivers, and careful signal processing are like **choosing the quieter café** — reducing the NESZ.  
- A poorly designed system, on the other hand, is like staying in the **packed party** 🥂🎶 — the NESZ is high, and only the loudest signals will ever be heard.  

But the story doesn’t stop there. Other factors also matter:  

- 📏 **Distance (Range):** If your friend moves further across the room, they’ll have to **speak louder** to reach you. If they sit closer, they can whisper. In radar, echoes from distant targets need to be stronger to stand out above the noise.  
- 👂 **Antenna Size (Ear Size):** If you had **huge elephant ears**, you could **tune in more directly** to your friend’s voice while filtering out some of the side chatter. Tiny ears, on the other hand, pick up everything equally and don’t help much in a noisy party. Bigger radar antennas work the same way — they can listen more selectively in one direction, boosting the received signal strength. _(And in radar, we don’t just listen — we also “shout” with the antenna. Bigger ears mean not only better hearing but also a more focused voice aimed at your friend, so they hear you more clearly too!)_  
- 🎚️ **Bandwidth (Focus):** Imagine **pricking up your ears** and listening with extreme focus to catch every detail of your friend’s words. You hear them more sharply — but you also let in more of the surrounding background hiss. In radar, higher bandwidth improves resolution, but it also increases the amount of noise that slips in.  

---

### 📊 Why Bandwidth Affects NESZ  

Here’s the twist:  

- Wider chirp bandwidth → **better resolution** (smaller range pixels)  
- But each smaller pixel returns **less energy** 🪞📉  
- So even though total **signal-to-noise-ratio (SNR)** doesn’t change, the **SNR per pixel drops** as pixels shrink.  

That’s why **NESZ gets worse as bandwidth increases**. ⚖️  

---

### 📏 What About the Synthetic Aperture?  

- Collecting more pulses (longer aperture) → **more signal energy** ✔️  
- But also produces **smaller cross-range pixels** ✔️  
- More signal, smaller slices → **SNR per pixel stays constant**  

So, **NESZ doesn’t depend on aperture length**, because the gains and losses cancel each other.  

---

### 💡 What NESZ Really Means  

NESZ tells us about **sensitivity** (how faint a signal can still be detected). But it says **nothing about resolution**. That’s why a **narrow bandwidth** looks great in an NESZ plot — low noise per pixel — but the images might have a very **poor resolution**. 📉📸  

So:  
- **Resolution** = how sharp the image is  
- **NESZ** = how well faint reflectors stand out  
- Good SAR design means balancing both. ⚖️  

### 📉 Exploring NESZ  

This simulation lets you explore how the **Noise Equivalent Sigma Zero (NESZ)** depends on radar design choices.  
- 📏 **Range** (x-axis) – farther targets are harder to hear; weak echoes fade with distance.  
- 🎚️ **Bandwidth** (y-axis) – sharper detail, but more noise sneaks in.  

Other system parameters can be adjusted with the sliders:  
- 👂 **Antenna size** – bigger “ears” mean higher gain and more selectivity.  
- ⏱️ **Collection duration** – the longer we listen, the more signal we can average out of the noise.  
- 📡 **Transmit power** – a stronger “shout” makes faint echoes easier to detect.  

A **larger antenna** boosts gain on both **transmit and receive**, allowing the radar to see **farther** and pick up **fainter echoes**. That’s exactly why the **Gen4 antenna** is such a powerful upgrade for us. 📡✨  

👉 On the plot, notice how the NESZ changes with range and bandwidth as you tweak the sliders.  
- **Lower NESZ is better** ✅ — the radar can detect **weaker targets** that would otherwise drown in noise.  
- **Higher NESZ is worse** ❌ — only the loudest “shouting” targets stand out.  

🔎 **A note on the dB scale:** NESZ is usually shown in **decibels (dB)**, where smaller (more negative) values mean better sensitivity. Typical homogeneous surfaces like **grasslands or forests** reflect at around **–10 dB**. That means if our NESZ curve is above –10 dB, we risk **losing sight of those kinds of targets**.  

In [ ]:
def build_nesz_voila_app():
    """
    Voilà-friendly NESZ explorer.
    Adjustable: Transmit Power (W), Collection Duration (s), Antenna Length (m), Antenna Width (m).
    Usage:
        from IPython.display import display
        display(build_nesz_voila_app())
    """
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import FloatSlider, VBox, HBox, Output, Label, Layout

    # --- Constants (fixed) ---
    c = 3e8
    k = 1.38e-23        # Boltzmann's constant
    T = 290.0           # Noise temperature (K)
    F = 10**(5/10)      # Noise figure (linear) => 5 dB
    L = 1.0             # System losses
    freq = 9.65e9
    lamb = c / freq
    pulse_duration = 20e-6
    PRF = 5000.0
    orbit_height = 500e3   # 500 km orbit altitude
    velocity = 7.5e3       # m/s (LEO scale)

    # Axes for plot (independent of sliders)
    R_values = np.linspace(orbit_height + 1e3, 1000e3, 120)  # ~501 km to 1000 km
    B_values = np.linspace(50e6, 1200e6, 120)                 # 50–1200 MHz
    R_grid, B_grid = np.meshgrid(R_values, B_values)

    # Widgets
    w_pt = FloatSlider(description="Tx Power (W)", value=2000.0,
                       min=100.0, max=50_000.0, step=50.0, readout_format=".0f",
                       continuous_update=False)
    w_pt.style.description_width = 'initial'
    w_cd = FloatSlider(description="Collect (s)", value=5.0,
                       min=0.2, max=20.0, step=0.1, readout_format=".1f",
                       continuous_update=False)
    w_cd.style.description_width = 'initial'    
    w_len = FloatSlider(description="Ant L (m)", value=3.2,
                        min=0.1, max=10.0, step=0.05, readout_format=".2f",
                        continuous_update=False)
    w_len.style.description_width = 'initial'
    w_wid = FloatSlider(description="Ant W (m)", value=0.4,
                        min=0.1, max=10.0, step=0.05, readout_format=".2f",
                        continuous_update=False)
    w_wid.style.description_width = 'initial'

    out = Output()

    def compute_and_plot(*_):
        Pt = w_pt.value
        collection_duration = w_cd.value
        ant_length = w_len.value
        ant_width = w_wid.value

        # Antenna gain from physical aperture (uniform illumination approximation)
        antenna_area = ant_length * ant_width
        G_linear = 4 * np.pi * antenna_area / (lamb ** 2)

        # Geometry terms
        N_pulses = max(int(PRF * collection_duration), 1)
        aperture_length = velocity * collection_duration

        # Incidence angle and safety floors
        cos_i = np.clip(orbit_height / R_grid, -1.0, 1.0)
        incidence_angle = np.arccos(cos_i)
        sin_i = np.sin(incidence_angle)
        sin_i = np.maximum(sin_i, 1e-9)

        # Chirp processing gain
        G_chirp = B_grid * pulse_duration

        # Resolutions
        # Ground-range resolution: Δrg = c / (2B * sin(incidence))
        delta_r = c / (2.0 * B_grid) * (1.0 / sin_i)

        # Cross-range (spotlight-like small-angle approx using aperture length):
        # Beamwidth ~ 2*atan((D/2)/R), use D = aperture_length
        theta = 2.0 * np.arctan((aperture_length / 2.0) / R_grid)
        theta = np.maximum(theta, 1e-9)
        delta_cr = lamb / (2.0 * theta)

        # NESZ (normalized backscatter threshold where echo power ~ noise power)
        NESZ = ((4.0 * np.pi) ** 3 * k * T * B_grid * F * L * (R_grid ** 4)) / (
               Pt * (G_linear ** 2) * (lamb ** 2) * G_chirp * N_pulses * delta_r * delta_cr)

        NESZ_dB = 10.0 * np.log10(NESZ)

        with out:
            out.clear_output(wait=True)
            plt.figure(figsize=(8, 6))
            levels = np.linspace(np.nanmin(NESZ_dB), np.nanmax(NESZ_dB), 40)
            cf = plt.contourf(R_values / 1e3, B_values / 1e6, NESZ_dB, levels=levels, cmap="jet")
            cb = plt.colorbar(cf)
            cb.set_label("NESZ (dB)")
            plt.xlabel("Slant Range R (km)")
            plt.ylabel("Chirp Bandwidth (MHz)")
            plt.title(
                f"NESZ vs. Range & Bandwidth\n"
                f"Pt={Pt:.0f} W, Collect={collection_duration:.1f} s, Ant={ant_length:.2f}×{ant_width:.2f} m, "
                f"PRF={PRF/1e3:.1f} kHz, τ={pulse_duration*1e6:.0f} µs"
            )
            plt.tight_layout()
            plt.show()

    # Wire updates
    for w in (w_pt, w_cd, w_len, w_wid):
        w.observe(compute_and_plot, names="value")

    # Initial draw
    compute_and_plot()

    controls = VBox([
        HBox([w_pt, w_cd]),
        HBox([w_len, w_wid])
    ])
    return VBox([controls, out])

from IPython.display import display
nesz_app = build_nesz_voila_app()
display(nesz_app)